In [18]:
import pandas as pd
import csv

file_tipi_gov = "tipi_gov.xlsx"
wb_tipi_gov = pd.ExcelFile(file_tipi_gov)

df_tipi = wb_tipi_gov.parse(header=7)
df_tipi.columns = ['ncm', 'exception', 'desc', 'ipi']

df_tipi = df_tipi.astype({'exception':'str'})
df_tipi["exception"] = df_tipi["exception"].str.zfill(2)

for i, row in df_tipi.iterrows():
    elements = str(row["ncm"]).split(".")
    if len(elements[0]) % 2 != 0:
        elements[0] = "0" + elements[0]
    df_tipi.at[i,"ncm"] = "".join(elements)

    if row["exception"] == "nan":
        df_tipi.at[i,"exception"] = ""

for i, row in df_tipi.iterrows():
    ncm = row["ncm"]
    if len(ncm) != 8:
        continue

    concat_desc = ""
    parts = [ncm[0:4],ncm[0:5],ncm[0:6],ncm[0:7],ncm[0:8]]
    for part in parts:
        result = df_tipi.query(f"ncm =='{part}'")
        if not result.empty:
            part_name = result["desc"].iloc[0]
            concat_desc = f"{concat_desc} {part_name}"
    df_tipi.at[i, "concat_desc"] = concat_desc[1:-1]

df_tipi = df_tipi.loc[df_tipi['ncm'].str.len() == 8]

In [20]:
df = pd.DataFrame()

# df["id"] = f"ncm_{df_tipi['ncm']}"

# df["id"] = "ncm_" + df_tipi['ncm']

for i, row in df_tipi.iterrows():
    id = f"ncm_{row['ncm']}"
    if row['exception'] != "":
        id = f"{id}_{row['exception']}"
    df.at[i, "id"] = id

    df.at[i, "code"] = f"{row['ncm'][:4]}.{row['ncm'][4:6]}.{row['ncm'][6:8]}"

    df.at[i, "exception"] = row['exception']

    df.at[i, "name"] = row['concat_desc']


df.to_csv('out.csv', quoting=csv.QUOTE_ALL,index=False)